In [10]:
import os
import tkinter as tk
from tkinter import filedialog
import xlwings as xw
from pathlib import Path
import shutil, tempfile
import pandas as pd

def select_file(file_name):
    root = tk.Tk()
    root.withdraw()
    root.attributes('-topmost', True)
    file_path = filedialog.askopenfilename(filetypes=[("Excel file", "*.xlsb *.xlsx")],title=f"Select {file_name} file", parent=root)
    return file_path

def open_astemporary_local(original_path: str) -> Path:
    original = Path(original_path)
    temp_dir = Path(tempfile.gettempdir())
    temp_copy = temp_dir / original.name
    shutil.copy2(original, temp_copy)
    return temp_copy


#select wb1, new PL, new BS
file_names = ["WB1 excel", "New Profit and Loss", "New Balance Sheet"]
wb1_path = select_file(file_names[0])
new_pl_path = select_file(file_names[1])
#new_bs_path = select_file(file_names[2])

#Check if app instance is running. If yes, then close it
for app in xw.apps:
    print(app)
    for book in app.books:
        print(book.fullname)
        if book.fullname.lower() == str(wb1_path).lower():
            book.close(save_changes=False)
    app.quit

# Open workbook
try:
    app = xw.App(visible=False)
    wb1 = app.books.open(open_astemporary_local(wb1_path))
    last_row = input("Enter last row number for data range in WB1: ")
    header_range = f"C5:AM5"
    value_range = f"C8:AM{last_row}"
    pl_header = wb1.sheets["1. PL Import"].range(header_range).value
    pl_values = wb1.sheets["1. PL Import"].range(value_range).value

    # rename the first column to 'Account'
    pl_header[0] = "Account"

    #create a dataframe from the PL data
    old_pl = pd.DataFrame(pl_values, columns=pl_header)

    #use index location in strip and lower the account names
    old_pl.iloc[:, 0] = old_pl.iloc[:,0].str.strip().str.lower()
    #print(old_pl.iloc[:,0])

    old_pl_unpivot = old_pl.melt(
        id_vars=["Account"],
        var_name="Month",
        value_name="Amount"
    )

    old_pl_unpivot["Month"] = pd.to_datetime(old_pl_unpivot["Month"], format="%Y-%m-%d %H:%M:%S").dt.strftime("%B-%Y")
    old_pl_unpivot["SortKey"] = range(len(old_pl_unpivot))

    account_and_type = pd.DataFrame(columns=["account name", "type"])
    #get account type from working PL
    try:
        account_types = {
            "revenue" : "D8:D57",
            "revenue_adj" : "D75:D102",
            "cogs" : "D117:D141",
            "cogs_others" : "D147:D182",
            "expenses_media" : "D189:D213",
            "expenes_general" : "D218:D283",
            "expenses_staffing" : "D286:D309"
        }

    except Exception as e:
        print(f"Error in getting account type: {e}")

except Exception as e:
    print(f"error {e}")


<App [excel] 2292>
G:\.shortcut-targets-by-id\1rElEdbhz1g109ZYRpS_uYNunrKhaqgo4\Clients\HoundsBay - Rich and Sam\Strategic Valuation\1. HoundsBay - Master MCOA V7.01 - 11.19.2025.xlsb


: 

: 

In [9]:
for item in account_types.values:
    accounts = wb1.sheets['3. Working P&L'].range(item)
print(accounts)

NameError: name 'account_types' is not defined